# Model

### package load

In [1]:
import datetime
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import time
from tqdm import tqdm
import numpy as np
import warnings
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.autograd import Variable

from model import convert_bn_to_instancenorm, convert_bn_to_evonorm, convert_bn_to_groupnorm, DeepLabHead 
from helpers import AverageMeter, ProgressMeter, iouCalc, visim, vislbl
from labels import labels

### device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#device = 'cpu'

### data load

In [4]:
npzfile = np.load('data_origin.npz')

train_x = npzfile['train_x']
train_y = npzfile['train_y']
test_x = npzfile['test_x']
test_y = npzfile['test_y']

npzfile.close()

### DeepLab v3 ResNet50 

In [5]:
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False).to(device)
         # dlv3 backbone resnoet 50 :: output chnnel: 2048 --> DL input channel :: 2048
model.classifier = DeepLabHead(2048, 12).to(device) # 12 = class num

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2) 

In [6]:
# Initialize metrics
best_miou = 0.0
metrics = {'train_loss' : [],
           'train_acc' : [],
           'test_acc' : [],
           'test_loss' : [],
           'miou' : []}
start_epoch = 0

### Label 

In [7]:
# Create list of class names
  # classLabels: label명
  # validClasses: label에 해당하는 id값

classLabels = []
for label in labels:
    if label.name not in classLabels:
        classLabels.append(label.name)
classLabels.append('void')

validClasses = list(
    np.unique(
        [label.id for label in labels 
         if label.id >= 0
        ] + [11]
    )
)

### train

In [9]:
## parameter

batch_size = 2
num_epoch = 5

result_dir = os.getcwd() + '/result/' + 'only_weight/'

In [10]:
from torch.utils.data import TensorDataset, DataLoader

   # X: 0~255, RGB값 가지는 이미지 데이터 (560, 720, 960, 3)
   # Y: 0~11 label값 가지는 픽셀당 클래스 데이터 (560, 720, 960)
X = torch.tensor(train_x, dtype=torch.float32)
Y = torch.tensor(train_y, dtype=torch.long)

data = TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)
train_data = DataLoader(data, batch_size=batch_size, shuffle=True)

In [2]:
# Create class weight
unique, counts = np.unique(train_y.flatten(), return_counts=True)

max_num = max(counts)
weights = [max_num/num for num in counts]

class_weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights, ignore_index=12)

In [3]:
class_weights

tensor([28.7103, 26.2137, 53.3813, 39.6513,  1.1817, 20.1356,  4.0933,  1.0000,
         5.8569,  1.8502,  2.5995,  9.8069])

In [14]:
model.train()
res = X.shape[1] * X.shape[2] # 720 * 960

for epoch in range(num_epoch):
    
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')
    
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    progress = ProgressMeter(
        len(train_data),
        [loss_running, acc_running],
        prefix="Train, epoch: [{}]".format(epoch))
    
    batch_loss = 0.0
    for batch, (x, y) in enumerate(tqdm(train_data, total=len(train_data))):
        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(x)
        outputs = outputs['out']
        preds = torch.argmax(outputs, 1)
        
        # cross-entropy loss
        loss = criterion(outputs, y)

        # backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))
        
        nvoid = int((y==12).sum()) 
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
        
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)
        
        progress.display(epoch)
        
        
    scheduler.step(loss_running.avg)
    miou = iou.outputScores()
    
    # save checkpoint
    now = datetime.datetime.now()
    now_time = now.strftime('%y%m%d%H')
    
    # save path
    if not os.path.isdir(result_dir):
        os.makedirs(result_dir)
    
    save_path = result_dir
    
    with open(save_path + 'train_logs.csv', 'a') as epochs:
            epochs.write('{}, {:.4f}, {:.4f}, {:.4f}\n'.format(
                    epoch+1, loss_running.avg, acc_running.avg, miou))
            
    # Save model to file
    torch.save({
        'epoch' : epoch + 1,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_miou': best_miou,
        'metrics': metrics,
        }, save_path + now_time + '_E' + str(epoch+1) + '_checkpoint.pth.tar')
    
    # Save best model to file
    if miou > best_miou:
        print('mIoU improved from {:.4f} to {:.4f}.'.format(best_miou, miou))
        best_miou = miou
        
    print('epoch ', epoch)
    print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

  0%|▎                                                                               | 1/280 [00:16<1:18:57, 16.98s/it]

Train, epoch: [0][  0/280]	Loss 2.5660e+00 (2.5660e+00)	Accuracy 0.059 (0.059)


  1%|▌                                                                               | 2/280 [00:33<1:17:53, 16.81s/it]

Train, epoch: [0][  0/280]	Loss 2.5356e+00 (2.5508e+00)	Accuracy 0.084 (0.072)


  1%|▊                                                                               | 3/280 [00:50<1:17:15, 16.73s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████▋| 279/280 [1:19:13<00:16, 16.97s/it]

Train, epoch: [4][  4/280]	Loss 2.0562e-01 (2.5707e-01)	Accuracy 0.924 (0.913)


100%|██████████████████████████████████████████████████████████████████████████████| 280/280 [1:19:30<00:00, 17.04s/it]

Train, epoch: [4][  4/280]	Loss 3.5145e-01 (2.5741e-01)	Accuracy 0.909 (0.913)
classes           IoU
---------------------
Pole          : 0.389
SignSymbol    : 0.604
Bicyclist     : 0.730
Pedestrian    : 0.601
Building      : 0.855
Fence         : 0.734
Pavement      : 0.837
Road          : 0.953
Car           : 0.876
Sky           : 0.906
Tree          : 0.817
---------------------
Mean IoU      : 0.755
---------------------


mIoU improved from 0.7348 to 0.7547.
epoch  4
loss : 0.2574   acc : 0.9131   miou : 0.7547


### test

In [15]:
X = torch.tensor(test_x, dtype=torch.float32)
Y = torch.tensor(test_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

test_data = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

save_path = result_dir
result = sorted(os.listdir(save_path), reverse=True)

In [16]:
checkpoint = torch.load(save_path + result[1]) # train, val 결과보고 결정 (overfitting 시작되기 전)
model.load_state_dict(checkpoint['model_state_dict'], strict=True)

<All keys matched successfully>

In [17]:
batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
progress = ProgressMeter(
    len(test_data),
    [batch_time, data_time],
    prefix='Predict: ')
loss_running = AverageMeter('Loss', ':.4e')
acc_running = AverageMeter('Accuracy', ':.3f')
    
iou = iouCalc(classLabels, validClasses, voidClass = 11)

model.eval()

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(test_data, total=len(test_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()

print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

100%|██████████████████████████████████████████████████████████████████████████████████| 71/71 [07:11<00:00,  6.07s/it]

classes           IoU
---------------------
Pole          : 0.349
SignSymbol    : 0.524
Bicyclist     : 0.700
Pedestrian    : 0.575
Building      : 0.828
Fence         : 0.724
Pavement      : 0.815
Road          : 0.953
Car           : 0.850
Sky           : 0.906
Tree          : 0.814
---------------------
Mean IoU      : 0.731
---------------------
loss : 0.3729 acc : 0.9041 miou : 0.7307
